In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext tensorboard

In [ ]:

pip install tensorflow-federated

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization,GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score,accuracy_score
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras import regularizers
import os
import sklearn.metrics
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import keras as ks
from sklearn.preprocessing import MinMaxScaler
from keras import layers
from tensorflow.keras.models import Model
#import tensorflow_federated as tff

In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:
dataset1 = pd.read_csv('/content/drive/MyDrive/4144.csv', index_col='ts', parse_dates=True)
dataset2 = pd.read_csv('/content/drive/MyDrive/4147.csv', index_col='ts', parse_dates=True)
dataset3 = pd.read_csv('/content/drive/MyDrive/4135.csv', index_col='ts', parse_dates=True)
dataset4 = pd.read_csv('/content/drive/MyDrive/4137.csv', index_col='ts', parse_dates=True)

In [ ]:
dataset1

In [ ]:
num_epochs =30
num_rounds=4

timesteps = 100


In [ ]:
def split_data_chronologically(data, test_ratio=0.2, val_ratio=0.2, target_column='anomaly_label'):
    total_samples = len(data)
    test_split_idx = int(total_samples * (1 - test_ratio))
    val_split_idx = int(test_split_idx * (1 - val_ratio / (1 - test_ratio)))

    train_data = data.iloc[:val_split_idx]
    val_data = data.iloc[val_split_idx:test_split_idx]
    test_data = data.iloc[test_split_idx:]

    train_labels = train_data[target_column].values.astype('float32')
    val_labels = val_data[target_column].values.astype('float32')
    test_labels = test_data[target_column].values.astype('float32')

    train_data = train_data.drop(columns=[target_column]).astype('float32')
    test_data = test_data.drop(columns=[target_column]).astype('float32')
    val_data = val_data.drop(columns=[target_column]).astype('float32')

    return train_data, val_data, test_data, train_labels, test_labels, val_labels

In [ ]:
def create_tf_dataset(features, labels, time_steps=timesteps, batch_size=32, shuffle=False):
    dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=features,
        targets=labels,
        sequence_length=time_steps,
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=batch_size,
    )
    return dataset

In [ ]:

def preprocess(data, labels, time_steps=timesteps, batch_size=128):
    # Cast data and labels to float32
    data = tf.cast(data, tf.float32)
    labels = tf.cast(labels, tf.float32)

    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.window(time_steps, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(time_steps), y.batch(time_steps))))
    dataset = dataset.batch(batch_size)
    return dataset.repeat(1)#.batch(batch_size).map(batch_format_fn).prefeth(tf.data.experimental.AUTOTUNE)


In [ ]:
METRICS = [
      ks.metrics.TruePositives(name='tp'),
      ks.metrics.FalsePositives(name='fp'),
      ks.metrics.TrueNegatives(name='tn'),
      ks.metrics.FalseNegatives(name='fn'),
      ks.metrics.BinaryAccuracy(name='accuracy'),
      ks.metrics.Precision(name='precision'),
      ks.metrics.Recall(name='recall'),
      ks.metrics.AUC(name='auc'),
      ks.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
def setup_data(dataset):
    # Split data chronologically
    train_data, val_data, test_data, train_labels, test_labels, val_labels = split_data_chronologically(dataset)

    # Convert data to numpy arrays
    train_data = train_data.to_numpy().astype('float32')
    val_data = val_data.to_numpy().astype('float32')
    test_data = test_data.to_numpy().astype('float32')

    # Standardize features
    scaler = StandardScaler()
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
    val_data = scaler.transform(val_data)

    return train_data, val_data, test_data, train_labels, test_labels, val_labels


In [ ]:
def preprocess2(data, labels, time_steps=timesteps, batch_size=128):
     dataset= create_tf_dataset(data,labels, time_steps=time_steps, batch_size=batch_size)

     return dataset.repeat(num_epochs)

In [ ]:
train_data1, val_data1, test_data1, train_labels1, test_labels1, val_labels1 = setup_data(dataset1)
train_data2, val_data2, test_data2, train_labels2, test_labels2, val_labels2 = setup_data(dataset2)
train_data3, val_data3, test_data3, train_labels3, test_labels3, val_labels3 = setup_data(dataset3)
train_data4, val_data4, test_data4, train_labels4, test_labels4, val_labels4 = setup_data(dataset4)

In [ ]:

num_features  = train_data1.shape[-1]

In [ ]:
client_ids = ['client1', 'client2', 'client3', 'client4']


In [ ]:

train_datasets =   [train_data1, train_data2, train_data3, train_data4]
train_labels = [train_labels1, train_labels2, train_labels3, train_labels4]
val_datasets=[val_data1, val_data2, val_data3, val_data4]
val_labels=[val_labels1, val_labels2, val_labels3, val_labels4]
test_datasets=[test_data1, test_data2, test_data3, test_data4]
test_labels =[test_labels1, test_labels2, test_labels3, test_labels4]

In [ ]:
class MyClientData(tff.simulation.datasets.ClientData):

    def __init__(self, client_ids, datasets):
        self._client_ids = client_ids
        self._datasets = datasets

    @property
    def client_ids(self):
        return self._client_ids

    def create_tf_dataset_for_client(self, client_id):
        return self._datasets[self._client_ids.index(client_id)]

    @property
    def element_type_structure(self):
        return (tf.TensorSpec(shape=(None, 100, 14), dtype=tf.float32), tf.TensorSpec(shape=(None, 100), dtype=tf.float32))

    def serializable_dataset_fn(self):
        return tf.function(lambda client_id: self.create_tf_dataset_for_client(client_id), input_signature=[tf.TensorSpec(shape=(), dtype=tf.string)])



In [ ]:
train_datasets = [preprocess2(train_datasets[i], train_labels[i]) for i in range(len(client_ids))]
val_datasets = [preprocess2(val_datasets[i], val_labels[i]) for i in range(len(client_ids))]
test_datasets = [preprocess2(test_datasets[i], test_labels[i]) for i in range(len(client_ids))]

In [ ]:

# Create your datasets


# Create ClientData objects
train_client_data = MyClientData(client_ids, train_datasets)
val_client_data = MyClientData(client_ids, val_datasets)
test_client_data = MyClientData(client_ids, test_datasets)

# Create lists of datasets
federated_train_data = [train_client_data.create_tf_dataset_for_client(client_id) for client_id in client_ids]
federated_val_data = [val_client_data.create_tf_dataset_for_client(client_id) for client_id in client_ids]
federated_test_data = [test_client_data.create_tf_dataset_for_client(client_id) for client_id in client_ids]



In [ ]:
federated_test_data[0].element_spec

(TensorSpec(shape=(None, None, 13), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

In [ ]:
def create_model():
   input_shape = (timesteps, num_features)
   model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=input_shape),
            tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),

            tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),

            tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),

            tf.keras.layers.LSTM(128, activation='relu'),  # Optional dense layer after flattening
            tf.keras.layers.Dropout(0.5),  # Optional dropout layer
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

   return model



In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 100, 32)           1280      
                                                                 
 max_pooling1d (MaxPooling1  (None, 50, 32)            0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 50, 32)            128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 50, 32)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 25, 64)            0

In [ ]:


def model_fn():
    keras_model = create_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=federated_test_data[0].element_spec,
        loss=tf.keras.losses.BinaryFocalCrossentropy(alpha=0.7, gamma=4),
        metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )


In [ ]:
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.0001)

def server_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.1)

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

In [ ]:
def set_sizing_environment():
  """Creates an environment that contains sizing information."""
  # Creates a sizing executor factory to output communication cost
  # after the training finishes. Note that sizing executor only provides an
  # estimate (not exact) of communication cost, and doesn't capture cases like
  # compression of over-the-wire representations. However, it's perfect for
  # demonstrating the effect of compression in this tutorial.
  sizing_factory = tff.framework.ExecutorFactory.create_executor()

  # TFF has a modular runtime you can configure yourself for various
  # environments and purposes, and this example just shows how to configure one
  # part of it to report the size of things.
  context = tff.framework.ExecutionContext(executor_fn=sizing_factory)
  tff.framework.set_default_context(context)

  return sizing_factory

In [ ]:
def format_size(size):
  """A helper function for creating a human-readable size."""
  size = float(size)
  for unit in ['B','KiB','MiB','GiB']:
    if size < 1024.0:
      return "{size:3.2f}{unit}".format(size=size, unit=unit)
    size /= 1024.0
  return "{size:.2f}{unit}".format(size=size, unit='TiB')

In [ ]:
print(training_process.initialize.type_signature.formatted_representation())

In [ ]:
# Step 5: Train Model
def train_federated_model(federated_train_data, federated_val_data, federated_test_data):

    #environment = set_sizing_environment()

    train_state = training_process.initialize()
    evaluation_state = evaluation_process.initialize()
    model_weights = training_process.get_model_weights(train_state)
    evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

    NUM_ROUNDS = 2
    for round_num in range(1, NUM_ROUNDS):
            result = training_process.next(train_state, federated_train_data)
            train_state = result.state
            train_metrics = result.metrics
            #size_info = environment.get_size_info()
            #broadcasted_bits = size_info.broadcast_bits[-1]
            #aggregated_bits = size_info.aggregate_bits[-1]
            print('round {:2d}, metrics={}'.format(round_num, train_metrics))# format_size(broadcasted_bits), format_size(aggregated_bits)))
            model_weights = training_process.get_model_weights(train_state)
            evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
            evaluation_output = evaluation_process.next(evaluation_state, federated_val_data)
            val_metrics = evaluation_output.metrics
            print('Round {:2d}, Validation Metrics: {}'.format(round_num, val_metrics))


    evaluation_test = evaluation_process.next(evaluation_state, federated_test_data)
    test_metrics = evaluation_test.metrics
    print('Round {:2d}, Test Metrics: {}'.format(NUM_ROUNDS, test_metrics))



In [ ]:
!rm -R /content/drive/MyDrive/logfed/*

log_dir = '/content/drive/MyDrive/logfed/original/'
summary_writer = tf.summary.create_file_writer(log_dir)

In [ ]:

# Step 5: Train Model
train_federated_model(federated_train_data, federated_val_data, federated_test_data)

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.96581334), ('precision', 0.9445769), ('recall', 0.92066574), ('loss', 0.024789102), ('num_examples', 49335150), ('num_batches', 385500)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round  1, Validation Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('binary_accuracy', 0.76909727), ('precision', 0.16666667), ('recall', 7.904576e-06), ('loss', 0.15645836), ('num_examples', 16437180), ('num_batches', 128460)])), ('total_rounds_metrics', OrderedDict([('binary_accuracy', 0.76909727), ('precision', 0.16666667), ('recall', 7.904576e-06), ('loss', 0.15645836), ('num_examples', 16437180), ('num_batches', 128460)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('fi

In [ ]:
NUM_ROUNDS = 3
for round_num in range(1, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))
  model_weights = training_process.get_model_weights(train_state)
  evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
  evaluation_output = evaluation_process.next(evaluation_state, federated_val_data)
  val_metrics = evaluation_output.metrics
  print('Round {:2d}, Validation Metrics: {}'.format(round_num, val_metrics))

evaluation_test = evaluation_process.next(evaluation_state, federated_test_data)
test_metrics = evaluation_test.metrics
print(test_metrics={test_metrics})

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_train_data)